# IPAD knockoffs

This tutorial generates knockoffs based on the [intertwined probabilistic factors decoupling (IPAD)](https://www.tandfonline.com/doi/full/10.1080/01621459.2019.1654878) method, described in the following paper

> Fan, Yingying, Jinchi Lv, Mahrad Sharifvaghefi, and Yoshimasa Uematsu. "IPAD: stable interpretable forecasting with knockoffs inference." Journal of the American Statistical Association 115, no. 532 (2020): 1822-1834.

As we will see shortly,

+ IPAD knockoffs are more powerful and much more efficient to generate than model-X MVR/ME/SDP knockoffs, if we can assume a low-dimensional factored model for the data matrix `X`. 
+ If `X` doesn't really assume a factored model, then IPAD knockoff's empirical FDR will be inflated, sometimes severely so.

Our comparison uses target FDR 10%,  Lasso coefficient difference statistic, and we explore 3 ways (ER, GR, VR) to choose the number of latent factors `r` for IPAD knockoffs.

In [1]:
# load packages
using Knockoffs
using LinearAlgebra
using Random
using StatsBase
using Statistics
using ToeplitzMatrices
using Distributions
using Random
using CSV, DataFrames

# some helper functions to compute power and empirical FDR
function TP(correct_groups, signif_groups)
    return length(signif_groups ∩ correct_groups) / max(1, length(correct_groups))
end
function TP(correct_groups, β̂, groups)
    signif_groups = get_signif_groups(β̂, groups)
    return TP(correct_groups, signif_groups)
end
function FDR(correct_groups, signif_groups)
    FP = length(signif_groups) - length(signif_groups ∩ correct_groups) # number of false positives
    return FP / max(1, length(signif_groups))
end
function FDR(correct_groups, β̂, groups)
    signif_groups = get_signif_groups(β̂, groups)
    return FDR(correct_groups, signif_groups)
end
function get_signif_groups(β, groups)
    correct_groups = Int[]
    for i in findall(!iszero, β)
        g = groups[i]
        g ∈ correct_groups || push!(correct_groups, g)
    end
    return correct_groups
end

[ Info: Precompiling Knockoffs [878bf26d-0c49-448a-9df5-b057c815d613]


get_signif_groups (generic function with 1 method)

## Test1: Simulate under factored model

This is design 1 of [IPAD: Stable Interpretable Forecasting with Knockoffs Inference](https://www.tandfonline.com/doi/abs/10.1080/01621459.2019.1654878?journalCode=uasa20). 

+ Here $n=500, p=1000$
+ ``X = F\Lambda' + \sqrt{r\theta}E`` where ``r = 3, \theta=1`` and ``F, \Lambda, E`` are ``n \times r, p \times r`, and ``n \times p`` iid gaussian matrices. 
+ ``y = X\beta + \sqrt{c}\epsilon`` where ``c = 0.2``
+ 50 causal variables with $\beta_i = A = 0.1$
+ For MVR/ME/SDP, we generate 2nd order knockoffs by estimating a shrinked covariance matrix

In [2]:
function compare_ipad(nsims)
    n = 500 # number of samples
    p = 1000 # number of covariates
    m = 1    # number of knockoffs per variable
    k = 50   # number of causal variables
    rtrue = 3 # true rank
    A = 0.1 # causal beta
    θ = 1
    c = 0.2 # some noise term for simulating y    
    
    sdp_powers, sdp_fdrs, sdp_times = 0.0, 0.0, 0.0
    me_powers, me_fdrs, me_times = 0.0, 0.0, 0.0
    mvr_powers, mvr_fdrs, mvr_times = 0.0, 0.0, 0.0
    ipad_er_powers, ipad_er_fdrs, ipad_er_times = 0.0, 0.0, 0.0
    ipad_gr_powers, ipad_gr_fdrs, ipad_gr_times = 0.0, 0.0, 0.0
    ipad_ve_powers, ipad_ve_fdrs, ipad_ve_times = 0.0, 0.0, 0.0

    for seed in 1:nsims
        # simulate X
        Random.seed!(seed)
        F = randn(n, rtrue)
        Λ = randn(p, rtrue)
        C = F * Λ'
        E = randn(n, p)
        X = C + sqrt(rtrue * θ) * E

        # simulate y
        Random.seed!(seed)
        βtrue = zeros(p)
        βtrue[1:k] .= rand(-1:2:1, k) .* A
        shuffle!(βtrue)
        ϵ = randn(n)
        y = X * βtrue + sqrt(c) .* ϵ
        μ = zeros(p)
        correct_snps = findall(!iszero, βtrue)

        # ipad with ER
        Random.seed!(seed)
        ipad_er_t = @elapsed ipad_er = ipad(X, r_method = :er, m = m)
        ipad_er_ko_filter = fit_lasso(y, ipad_er)
        discovered_snps = findall(!iszero, ipad_er_ko_filter.βs[3])
        ipad_er_power = round(TP(correct_snps, discovered_snps), digits=3)
        ipad_er_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        #ipad with GR
        Random.seed!(seed)
        ipad_gr_t = @elapsed ipad_gr = ipad(X, r_method = :gr, m = m)
        ipad_gr_ko_filter = fit_lasso(y, ipad_gr)
        discovered_snps = findall(!iszero, ipad_gr_ko_filter.βs[3])
        ipad_gr_power = round(TP(correct_snps, discovered_snps), digits=3)
        ipad_gr_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        # ipad with VE
        Random.seed!(seed)
        ipad_ve_t = @elapsed ipad_ve = ipad(X, r_method = :ve, m = m)
        ipad_ve_ko_filter = fit_lasso(y, ipad_ve)
        discovered_snps = findall(!iszero, ipad_ve_ko_filter.βs[3])
        ipad_ve_power = round(TP(correct_snps, discovered_snps), digits=3)
        ipad_ve_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        # ME knockoffs
        Random.seed!(seed)
        me_t = @elapsed me = modelX_gaussian_knockoffs(X, :maxent, m = m)
        me_ko_filter = fit_lasso(y, me)
        discovered_snps = findall(!iszero, me_ko_filter.βs[3])
        me_power = round(TP(correct_snps, discovered_snps), digits=3)
        me_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        # MVR knockoffs
        Random.seed!(seed)
        mvr_t = @elapsed mvr = modelX_gaussian_knockoffs(X, :mvr, m = m)
        mvr_ko_filter = fit_lasso(y, mvr)
        discovered_snps = findall(!iszero, mvr_ko_filter.βs[3])
        mvr_power = round(TP(correct_snps, discovered_snps), digits=3)
        mvr_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        # SDP (ccd) knockoffs
        Random.seed!(seed)
        sdp_t = @elapsed sdp = modelX_gaussian_knockoffs(X, :sdp_ccd, m = m)
        sdp_ko_filter = fit_lasso(y, sdp)
        discovered_snps = findall(!iszero, sdp_ko_filter.βs[3])
        sdp_power = round(TP(correct_snps, discovered_snps), digits=3)
        sdp_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        # record results
        println("ipad ER: power = $ipad_er_power, FDR = $ipad_er_fdr, rank = $(ipad_er.r), time = $(round(ipad_er_t, digits=3))")
        println("ipad GR: power = $ipad_gr_power, FDR = $ipad_gr_fdr, rank = $(ipad_gr.r), time = $(round(ipad_gr_t, digits=3))")
        println("ipad VE: power = $ipad_ve_power, FDR = $ipad_ve_fdr, rank = $(ipad_ve.r), time = $(round(ipad_ve_t, digits=3))")
        println("ME: power = $me_power, FDR = $me_fdr, time = $(round(me_t, digits=3))")
        println("MVR: power = $mvr_power, FDR = $mvr_fdr, time = $(round(mvr_t, digits=3))")
        println("SDP: power = $sdp_power, FDR = $sdp_fdr, time = $(round(sdp_t, digits=3))\n\n")
        sdp_powers += sdp_power
        sdp_fdrs += sdp_fdr
        sdp_times += sdp_t
        me_powers += me_power
        me_fdrs += me_fdr
        me_times += me_t
        mvr_powers += mvr_power
        mvr_fdrs += mvr_fdr
        mvr_times += mvr_t
        ipad_er_powers += ipad_er_power
        ipad_er_fdrs += ipad_er_fdr
        ipad_er_times += ipad_er_t
        ipad_gr_powers += ipad_gr_power
        ipad_gr_fdrs += ipad_gr_fdr
        ipad_gr_times += ipad_gr_t
        ipad_ve_powers += ipad_ve_power
        ipad_ve_fdrs += ipad_ve_fdr
        ipad_ve_times += ipad_ve_t
    end
    
    # compute average
    ipad_er_powers /= nsims
    ipad_er_fdrs /= nsims
    ipad_er_times /= nsims
    ipad_gr_powers /= nsims
    ipad_gr_fdrs /= nsims
    ipad_gr_times /= nsims
    ipad_ve_powers /= nsims
    ipad_ve_fdrs /= nsims
    ipad_ve_times /= nsims
    me_powers /= nsims
    me_fdrs /= nsims
    me_times /= nsims
    mvr_powers /= nsims
    mvr_fdrs /= nsims
    mvr_times /= nsims
    sdp_powers /= nsims
    sdp_fdrs /= nsims
    sdp_times /= nsims
    
    # save in dataframe
    result = DataFrame(
        method=["IPAD-er", "IPAD-gr", "IPAD-ve", "ME", "MVR", "SDP"],
        power=[ipad_er_powers,ipad_gr_powers,ipad_ve_powers,me_powers,mvr_powers,sdp_powers], 
        FDR=[ipad_er_fdrs,ipad_gr_fdrs,ipad_ve_fdrs,me_fdrs,mvr_fdrs,sdp_fdrs], 
        time=[ipad_er_times,ipad_gr_times,ipad_ve_times,me_times,mvr_times,sdp_times]
    )

    return result
end

compare_ipad (generic function with 1 method)

In [3]:
nsims = 20
result = compare_ipad(nsims);

ipad ER: power = 0.98, FDR = 0.075, rank = 3, time = 0.45
ipad GR: power = 0.98, FDR = 0.075, rank = 3, time = 0.125
ipad VE: power = 1.0, FDR = 0.167, rank = 257, time = 0.108
ME: power = 0.0, FDR = 0.0, time = 2.748
MVR: power = 0.0, FDR = 0.0, time = 2.742
SDP: power = 0.48, FDR = 0.0, time = 11.437


ipad ER: power = 1.0, FDR = 0.265, rank = 3, time = 0.082
ipad GR: power = 1.0, FDR = 0.265, rank = 3, time = 0.041
ipad VE: power = 1.0, FDR = 0.123, rank = 263, time = 0.045
ME: power = 0.5, FDR = 0.0, time = 2.229
MVR: power = 0.4, FDR = 0.0, time = 2.769
SDP: power = 0.94, FDR = 0.078, time = 11.942


ipad ER: power = 0.96, FDR = 0.02, rank = 3, time = 0.139
ipad GR: power = 0.96, FDR = 0.02, rank = 3, time = 0.066
ipad VE: power = 0.98, FDR = 0.093, rank = 257, time = 0.072
ME: power = 0.66, FDR = 0.0, time = 2.306
MVR: power = 0.66, FDR = 0.0, time = 2.759
SDP: power = 0.8, FDR = 0.024, time = 11.682


ipad ER: power = 0.94, FDR = 0.041, rank = 3, time = 0.13
ipad GR: power = 0.9

In [4]:
# check average
@show result;

result = 6×4 DataFrame
 Row │ method   power    FDR      time
     │ String   Float64  Float64  Float64
─────┼───────────────────────────────────────
   1 │ IPAD-er    0.978  0.10215   0.133172
   2 │ IPAD-gr    0.978  0.10215   0.0811693
   3 │ IPAD-ve    0.972  0.0988    0.0707433
   4 │ ME         0.41   0.00465   2.3128
   5 │ MVR        0.361  0.0045    2.83926
   6 │ SDP        0.622  0.0207   11.5767


Summary
+ All methods control FDR (target = 10%) 
+ ER and GR method always find the correct rank (r = 3), while VE overestimates the rank
+ IPAD method has much better power compared to model-X knockoffs via ME/MVR/SDP construction
+ Surprisingly, ME/MVR has worse power than SDP
+ IPAD method is much more efficient to construct

## Test2: Try $X$ that's not a factored model

+ Here ``y \sim N(X\beta, 1)`` and ``X_i \sim N(0, \Sigma)`` where ``\Sigma`` is an AR(1) model.
+ 50 causal SNPs with ``\beta_i \sim \pm N(0, 0.5)``
+ For MVR/ME/SDP knockoffs, we assume the true ``\mu`` and ``\Sigma`` are available. 

In [11]:
function compare_ipad2(nsims)
    n = 500 # number of samples
    p = 1000 # number of covariates
    m = 1    # number of knockoffs per variable
    k = 50   # number of causal variables

    sdp_powers, sdp_fdrs, sdp_times = 0.0, 0.0, 0.0
    me_powers, me_fdrs, me_times = 0.0, 0.0, 0.0
    mvr_powers, mvr_fdrs, mvr_times = 0.0, 0.0, 0.0
    ipad_er_powers, ipad_er_fdrs, ipad_er_times = 0.0, 0.0, 0.0
    ipad_gr_powers, ipad_gr_fdrs, ipad_gr_times = 0.0, 0.0, 0.0
    ipad_ve_powers, ipad_ve_fdrs, ipad_ve_times = 0.0, 0.0, 0.0

    for seed in 1:nsims
        # simulate X
        Random.seed!(seed)
        Σ = simulate_AR1(p, a=3, b=1)
        μ = zeros(p)
        X = rand(MvNormal(μ, Σ), n)' |> Matrix
        zscore!(X, mean(X, dims=1), std(X, dims=1))

        # simulate y
        Random.seed!(seed)
        βtrue = zeros(p)
        βtrue[1:k] .= rand(-1:2:1, k) .* rand(Normal(0, 0.5), k)
        shuffle!(βtrue)
        ϵ = randn(n)
        y = X * βtrue + ϵ
        μ = zeros(p)
        correct_snps = findall(!iszero, βtrue)

        # ipad with ER
        Random.seed!(seed)
        ipad_er_t = @elapsed ipad_er = ipad(X, r_method = :er, m = m)
        ipad_er_ko_filter = fit_lasso(y, ipad_er)
        discovered_snps = findall(!iszero, ipad_er_ko_filter.βs[3])
        ipad_er_power = round(TP(correct_snps, discovered_snps), digits=3)
        ipad_er_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        #ipad with GR
        Random.seed!(seed)
        ipad_gr_t = @elapsed ipad_gr = ipad(X, r_method = :gr, m = m)
        ipad_gr_ko_filter = fit_lasso(y, ipad_gr)
        discovered_snps = findall(!iszero, ipad_gr_ko_filter.βs[3])
        ipad_gr_power = round(TP(correct_snps, discovered_snps), digits=3)
        ipad_gr_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        # ipad with VE
        Random.seed!(seed)
        ipad_ve_t = @elapsed ipad_ve = ipad(X, r_method = :ve, m = m)
        ipad_ve_ko_filter = fit_lasso(y, ipad_ve)
        discovered_snps = findall(!iszero, ipad_ve_ko_filter.βs[3])
        ipad_ve_power = round(TP(correct_snps, discovered_snps), digits=3)
        ipad_ve_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        # ME knockoffs
        Random.seed!(seed)
        me_t = @elapsed me = modelX_gaussian_knockoffs(X, :maxent, μ, Σ, m = m)
        me_ko_filter = fit_lasso(y, me)
        discovered_snps = findall(!iszero, me_ko_filter.βs[3])
        me_power = round(TP(correct_snps, discovered_snps), digits=3)
        me_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        # MVR knockoffs
        Random.seed!(seed)
        mvr_t = @elapsed mvr = modelX_gaussian_knockoffs(X, :mvr, μ, Σ,m = m)
        mvr_ko_filter = fit_lasso(y, mvr)
        discovered_snps = findall(!iszero, mvr_ko_filter.βs[3])
        mvr_power = round(TP(correct_snps, discovered_snps), digits=3)
        mvr_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        # SDP (ccd) knockoffs
        Random.seed!(seed)
        sdp_t = @elapsed sdp = modelX_gaussian_knockoffs(X, :sdp_ccd, μ, Σ, m = m)
        sdp_ko_filter = fit_lasso(y, sdp)
        discovered_snps = findall(!iszero, sdp_ko_filter.βs[3])
        sdp_power = round(TP(correct_snps, discovered_snps), digits=3)
        sdp_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        # record results
        println("ipad ER: power = $ipad_er_power, FDR = $ipad_er_fdr, rank = $(ipad_er.r), time = $(round(ipad_er_t, digits=3))")
        println("ipad GR: power = $ipad_gr_power, FDR = $ipad_gr_fdr, rank = $(ipad_gr.r), time = $(round(ipad_gr_t, digits=3))")
        println("ipad VE: power = $ipad_ve_power, FDR = $ipad_ve_fdr, rank = $(ipad_ve.r), time = $(round(ipad_ve_t, digits=3))")
        println("ME: power = $me_power, FDR = $me_fdr, time = $(round(me_t, digits=3))")
        println("MVR: power = $mvr_power, FDR = $mvr_fdr, time = $(round(mvr_t, digits=3))")
        println("SDP: power = $sdp_power, FDR = $sdp_fdr, time = $(round(sdp_t, digits=3))\n\n")
        sdp_powers += sdp_power
        sdp_fdrs += sdp_fdr
        sdp_times += sdp_t
        me_powers += me_power
        me_fdrs += me_fdr
        me_times += me_t
        mvr_powers += mvr_power
        mvr_fdrs += mvr_fdr
        mvr_times += mvr_t
        ipad_er_powers += ipad_er_power
        ipad_er_fdrs += ipad_er_fdr
        ipad_er_times += ipad_er_t
        ipad_gr_powers += ipad_gr_power
        ipad_gr_fdrs += ipad_gr_fdr
        ipad_gr_times += ipad_gr_t
        ipad_ve_powers += ipad_ve_power
        ipad_ve_fdrs += ipad_ve_fdr
        ipad_ve_times += ipad_ve_t
    end
    
    # compute average
    ipad_er_powers /= nsims
    ipad_er_fdrs /= nsims
    ipad_er_times /= nsims
    ipad_gr_powers /= nsims
    ipad_gr_fdrs /= nsims
    ipad_gr_times /= nsims
    ipad_ve_powers /= nsims
    ipad_ve_fdrs /= nsims
    ipad_ve_times /= nsims
    me_powers /= nsims
    me_fdrs /= nsims
    me_times /= nsims
    mvr_powers /= nsims
    mvr_fdrs /= nsims
    mvr_times /= nsims
    sdp_powers /= nsims
    sdp_fdrs /= nsims
    sdp_times /= nsims
    
    # save in dataframe
    result = DataFrame(
        method=["IPAD-er", "IPAD-gr", "IPAD-ve", "ME", "MVR", "SDP"],
        power=[ipad_er_powers,ipad_gr_powers,ipad_ve_powers,me_powers,mvr_powers,sdp_powers], 
        FDR=[ipad_er_fdrs,ipad_gr_fdrs,ipad_ve_fdrs,me_fdrs,mvr_fdrs,sdp_fdrs], 
        time=[ipad_er_times,ipad_gr_times,ipad_ve_times,me_times,mvr_times,sdp_times]
    )

    return result
end

compare_ipad2 (generic function with 1 method)

In [12]:
nsims = 20
result = compare_ipad2(nsims);

ipad ER: power = 0.0, FDR = 0.0, rank = 496, time = 0.139
ipad GR: power = 0.64, FDR = 0.179, rank = 1, time = 0.116
ipad VE: power = 0.62, FDR = 0.205, rank = 217, time = 0.079
ME: power = 0.28, FDR = 0.0, time = 1.998
MVR: power = 0.26, FDR = 0.0, time = 2.878
SDP: power = 0.4, FDR = 0.0, time = 7.752


ipad ER: power = 0.5, FDR = 0.242, rank = 1, time = 0.042
ipad GR: power = 0.5, FDR = 0.242, rank = 1, time = 0.063
ipad VE: power = 0.5, FDR = 0.242, rank = 219, time = 0.044
ME: power = 0.0, FDR = 0.0, time = 1.889
MVR: power = 0.2, FDR = 0.0, time = 2.765
SDP: power = 0.38, FDR = 0.05, time = 7.801


ipad ER: power = 0.58, FDR = 0.216, rank = 1, time = 0.046
ipad GR: power = 0.58, FDR = 0.216, rank = 1, time = 0.116
ipad VE: power = 0.62, FDR = 0.244, rank = 217, time = 0.15
ME: power = 0.5, FDR = 0.107, time = 2.102
MVR: power = 0.48, FDR = 0.077, time = 2.801
SDP: power = 0.0, FDR = 0.0, time = 7.656


ipad ER: power = 0.46, FDR = 0.148, rank = 480, time = 0.048
ipad GR: power = 

In [13]:
@show result;

result = 6×4 DataFrame
 Row │ method   power    FDR      time
     │ String   Float64  Float64  Float64
─────┼──────────────────────────────────────
   1 │ IPAD-er    0.382  0.13535  0.0554262
   2 │ IPAD-gr    0.617  0.26555  0.0707459
   3 │ IPAD-ve    0.62   0.2813   0.0648921
   4 │ ME         0.382  0.07175  2.02233
   5 │ MVR        0.383  0.06555  2.73484
   6 │ SDP        0.35   0.0549   7.8911


Summary

+ IPAD methods have slightly~pretty inflated FDR (target = 10%) 
+ When $X$ does not follow the factored model, ER/GR/VE finds wildly differing ranks


## Test3: Simulate with gnomAD panel

+ Here we simulate ``X_i \sim N(0, \Sigma)`` where ``\Sigma`` is from the European [gnomAD LD panel](https://gnomad.broadinstitute.org/downloads#v2-linkage-disequilibrium). 
+ ``\Sigma`` can be downloaded and extract with the software [EasyLD.jl](https://github.com/biona001/EasyLD.jl). 

In [6]:
function compare_ipad3(nsims)
    # import panel
    datadir = "/Users/biona001/Benjamin_Folder/research/4th_project_PRS/group_knockoff_test_data"
    mapfile = CSV.read(joinpath(datadir, "Groups_2_127374341_128034347.txt"), DataFrame)
    groups = mapfile[!, :group]
    covfile = CSV.read(joinpath(datadir, "CorG_2_127374341_128034347.txt"), DataFrame)
    Σ = covfile |> Matrix{Float64}
    Σ = 0.99Σ + 0.01I #ensure PSD

    # test on smaller data
    idx = findlast(x -> x == 263, groups) # 263 is largest group with 192 members
    groups = groups[1:idx]
    Σ = Σ[1:idx, 1:idx]
    
    # simulation parameters
    n = 500 # number of samples
    p = size(Σ, 1) # number of covariates
    m = 1    # number of knockoffs per variable
    k = 50   # number of causal variables
    
    sdp_powers, sdp_fdrs, sdp_times = 0.0, 0.0, 0.0
    me_powers, me_fdrs, me_times = 0.0, 0.0, 0.0
    mvr_powers, mvr_fdrs, mvr_times = 0.0, 0.0, 0.0
    ipad_er_powers, ipad_er_fdrs, ipad_er_times = 0.0, 0.0, 0.0
    ipad_gr_powers, ipad_gr_fdrs, ipad_gr_times = 0.0, 0.0, 0.0
    ipad_ve_powers, ipad_ve_fdrs, ipad_ve_times = 0.0, 0.0, 0.0

    for seed in 1:nsims
        # simulate X
        Random.seed!(seed)
        X = rand(MvNormal(Σ), n)' |> Matrix
        zscore!(X, mean(X, dims=1), std(X, dims=1));

        # simulate y
        Random.seed!(seed)
        βtrue = zeros(p)
        βtrue[1:k] .= rand(-1:2:1, k) .* randn(k)
        shuffle!(βtrue)
        ϵ = randn(n)
        y = X * βtrue + ϵ
        μ = zeros(p)
        correct_snps = findall(!iszero, βtrue)

        # ipad with ER
        Random.seed!(seed)
        ipad_er_t = @elapsed ipad_er = ipad(X, r_method = :er, m = m)
        ipad_er_ko_filter = fit_lasso(y, ipad_er)
        discovered_snps = findall(!iszero, ipad_er_ko_filter.βs[3])
        ipad_er_power = round(TP(correct_snps, discovered_snps), digits=3)
        ipad_er_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        #ipad with GR
        Random.seed!(seed)
        ipad_gr_t = @elapsed ipad_gr = ipad(X, r_method = :gr, m = m)
        ipad_gr_ko_filter = fit_lasso(y, ipad_gr)
        discovered_snps = findall(!iszero, ipad_gr_ko_filter.βs[3])
        ipad_gr_power = round(TP(correct_snps, discovered_snps), digits=3)
        ipad_gr_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        # ipad with VE
        Random.seed!(seed)
        ipad_ve_t = @elapsed ipad_ve = ipad(X, r_method = :ve, m = m)
        ipad_ve_ko_filter = fit_lasso(y, ipad_ve)
        discovered_snps = findall(!iszero, ipad_ve_ko_filter.βs[3])
        ipad_ve_power = round(TP(correct_snps, discovered_snps), digits=3)
        ipad_ve_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        # ME knockoffs
        Random.seed!(seed)
        me_t = @elapsed me = modelX_gaussian_knockoffs(X, :maxent, m = m)
        me_ko_filter = fit_lasso(y, me)
        discovered_snps = findall(!iszero, me_ko_filter.βs[3])
        me_power = round(TP(correct_snps, discovered_snps), digits=3)
        me_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        # MVR knockoffs
        Random.seed!(seed)
        mvr_t = @elapsed mvr = modelX_gaussian_knockoffs(X, :mvr, m = m)
        mvr_ko_filter = fit_lasso(y, mvr)
        discovered_snps = findall(!iszero, mvr_ko_filter.βs[3])
        mvr_power = round(TP(correct_snps, discovered_snps), digits=3)
        mvr_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        # SDP (ccd) knockoffs
        Random.seed!(seed)
        sdp_t = @elapsed sdp = modelX_gaussian_knockoffs(X, :sdp_ccd, m = m)
        sdp_ko_filter = fit_lasso(y, sdp)
        discovered_snps = findall(!iszero, sdp_ko_filter.βs[3])
        sdp_power = round(TP(correct_snps, discovered_snps), digits=3)
        sdp_fdr = round(FDR(correct_snps, discovered_snps), digits=3)

        # record results
        println("ipad ER: power = $ipad_er_power, FDR = $ipad_er_fdr, rank = $(ipad_er.r), time = $(round(ipad_er_t, digits=3))")
        println("ipad GR: power = $ipad_gr_power, FDR = $ipad_gr_fdr, rank = $(ipad_gr.r), time = $(round(ipad_gr_t, digits=3))")
        println("ipad VE: power = $ipad_ve_power, FDR = $ipad_ve_fdr, rank = $(ipad_ve.r), time = $(round(ipad_ve_t, digits=3))")
        println("ME: power = $me_power, FDR = $me_fdr, time = $(round(me_t, digits=3))")
        println("MVR: power = $mvr_power, FDR = $mvr_fdr, time = $(round(mvr_t, digits=3))")
        println("SDP: power = $sdp_power, FDR = $sdp_fdr, time = $(round(sdp_t, digits=3))\n\n")
        sdp_powers += sdp_power
        sdp_fdrs += sdp_fdr
        sdp_times += sdp_t
        me_powers += me_power
        me_fdrs += me_fdr
        me_times += me_t
        mvr_powers += mvr_power
        mvr_fdrs += mvr_fdr
        mvr_times += mvr_t
        ipad_er_powers += ipad_er_power
        ipad_er_fdrs += ipad_er_fdr
        ipad_er_times += ipad_er_t
        ipad_gr_powers += ipad_gr_power
        ipad_gr_fdrs += ipad_gr_fdr
        ipad_gr_times += ipad_gr_t
        ipad_ve_powers += ipad_ve_power
        ipad_ve_fdrs += ipad_ve_fdr
        ipad_ve_times += ipad_ve_t
    end
    
    # compute average
    ipad_er_powers /= nsims
    ipad_er_fdrs /= nsims
    ipad_er_times /= nsims
    ipad_gr_powers /= nsims
    ipad_gr_fdrs /= nsims
    ipad_gr_times /= nsims
    ipad_ve_powers /= nsims
    ipad_ve_fdrs /= nsims
    ipad_ve_times /= nsims
    me_powers /= nsims
    me_fdrs /= nsims
    me_times /= nsims
    mvr_powers /= nsims
    mvr_fdrs /= nsims
    mvr_times /= nsims
    sdp_powers /= nsims
    sdp_fdrs /= nsims
    sdp_times /= nsims
    
    # save in dataframe
    result = DataFrame(
        method=["IPAD-er", "IPAD-gr", "IPAD-ve", "ME", "MVR", "SDP"],
        power=[ipad_er_powers,ipad_gr_powers,ipad_ve_powers,me_powers,mvr_powers,sdp_powers], 
        FDR=[ipad_er_fdrs,ipad_gr_fdrs,ipad_ve_fdrs,me_fdrs,mvr_fdrs,sdp_fdrs], 
        time=[ipad_er_times,ipad_gr_times,ipad_ve_times,me_times,mvr_times,sdp_times]
    )

    return result
end

compare_ipad3 (generic function with 1 method)

In [7]:
nsims = 20
result = compare_ipad3(nsims);

ipad ER: power = 0.4, FDR = 0.31, rank = 1, time = 0.083
ipad GR: power = 0.4, FDR = 0.31, rank = 1, time = 0.085
ipad VE: power = 0.38, FDR = 0.321, rank = 90, time = 0.115
ME: power = 0.34, FDR = 0.056, time = 13.224
MVR: power = 0.36, FDR = 0.053, time = 12.361
SDP: power = 0.32, FDR = 0.0, time = 58.465


ipad ER: power = 0.18, FDR = 0.25, rank = 1, time = 0.089
ipad GR: power = 0.18, FDR = 0.25, rank = 1, time = 0.089
ipad VE: power = 0.18, FDR = 0.308, rank = 91, time = 0.116
ME: power = 0.46, FDR = 0.233, time = 13.859
MVR: power = 0.44, FDR = 0.241, time = 13.163
SDP: power = 0.44, FDR = 0.043, time = 62.731


ipad ER: power = 0.32, FDR = 0.385, rank = 1, time = 0.095
ipad GR: power = 0.32, FDR = 0.385, rank = 1, time = 0.094
ipad VE: power = 0.36, FDR = 0.357, rank = 91, time = 0.099
ME: power = 0.38, FDR = 0.05, time = 13.979
MVR: power = 0.34, FDR = 0.0, time = 15.163
SDP: power = 0.36, FDR = 0.0, time = 62.336


ipad ER: power = 0.42, FDR = 0.4, rank = 1, time = 0.098
ipad 

In [19]:
@show result;

result = 6×4 DataFrame
 Row │ method   power    FDR      time
     │ String   Float64  Float64  Float64
─────┼───────────────────────────────────────
   1 │ IPAD-er    0.367  0.40725   0.0903964
   2 │ IPAD-gr    0.367  0.40725   0.0946635
   3 │ IPAD-ve    0.37   0.41525   0.100552
   4 │ ME         0.348  0.0936   14.2259
   5 │ MVR        0.344  0.0896   13.6386
   6 │ SDP        0.362  0.09235  60.8669


Summary
+ IPAD methods have extremely inflated FDR (target = 10%)